### CNN dimensions stuff

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import pandas as pd
import numpy as np
import sys,os,re,csv,pathlib

import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
c1 = nn.Conv2d(1, 8, kernel_size=(15,2))
m1 = nn.MaxPool2d(kernel_size = (1,2))
c2 = nn.Conv2d(8, 16, kernel_size = (1,2))
m2 = nn.MaxPool2d(kernel_size = (1,2))

cnn = nn.Sequential(c1,m1,c2,m2)

In [3]:
for z in range(12,17):
    print("####\n\nUSING LENGTH = ",z)
    x = torch.rand((1,1,15,z),dtype=torch.float32)
    print("Input: ",x.shape)
    conv1=c1(x)
    print("After Conv1:", conv1.shape)
    pool1=m1(conv1)
    print("After Pool1:", pool1.shape)
    conv2=c2(pool1)
    print("After Conv2:", conv2.shape)
    pool2=m2(conv2)
    print("After Pool2:", pool2.shape)
    if z<15:
        xx = 16*2
    elif z>=15:
        xx = 16*3
    lin = nn.Linear(xx,10)
    
    dense1= lin(pool2.view(-1,xx))
    print("Dense",dense1.shape)
    print("soft",dense1.softmax(1))
    print("arg",dense1.argmax(1))
    print("####\n\n")


####

USING LENGTH =  12
Input:  torch.Size([1, 1, 15, 12])
After Conv1: torch.Size([1, 8, 1, 11])
After Pool1: torch.Size([1, 8, 1, 5])
After Conv2: torch.Size([1, 16, 1, 4])
After Pool2: torch.Size([1, 16, 1, 2])
Dense torch.Size([1, 10])
soft tensor([[0.0929, 0.1150, 0.0869, 0.1024, 0.0870, 0.0853, 0.1370, 0.1060, 0.0725,
         0.1151]], grad_fn=<SoftmaxBackward>)
arg tensor([6])
####


####

USING LENGTH =  13
Input:  torch.Size([1, 1, 15, 13])
After Conv1: torch.Size([1, 8, 1, 12])
After Pool1: torch.Size([1, 8, 1, 6])
After Conv2: torch.Size([1, 16, 1, 5])
After Pool2: torch.Size([1, 16, 1, 2])
Dense torch.Size([1, 10])
soft tensor([[0.0969, 0.1168, 0.0839, 0.0816, 0.1263, 0.1539, 0.1043, 0.1046, 0.0517,
         0.0800]], grad_fn=<SoftmaxBackward>)
arg tensor([5])
####


####

USING LENGTH =  14
Input:  torch.Size([1, 1, 15, 14])
After Conv1: torch.Size([1, 8, 1, 13])
After Pool1: torch.Size([1, 8, 1, 6])
After Conv2: torch.Size([1, 16, 1, 5])
After Pool2: torch.Size([1, 16, 

### Parsing/AA index stuff/encoding

In [49]:
AAs=np.array(list('WFGAVILMPYSTNQCKRHDE'))
curPath=os.getcwd()
##AAidx_file='AAindexNormalized.txt' ## AA index reached AUC about 61% for L=14. Worse than AdaBoost
##AAidx_file='AtchleyFactors.txt'  ## Atchley factors work worse than using 544 AA index
AAidx_file='AAidx_PCA.txt' ## works like a charm!!!
gg=open(AAidx_file)
AAidx_Names=gg.readline().strip().split('\t') # Get PC1,... PC15
AAidx_Dict={} # Gets the values for each of the 15 features
i = 0
for ll in gg.readlines():
    if i ==4:
        print(ll)
    
    ll=ll.strip().split('\t')
    AA=ll[0]
    tag=0
    vv=[]
    for xx in ll[1:]:
        vv.append(float(xx))
    if tag==1:
        i+=1
        continue
    i+=1
    AAidx_Dict[AA]=vv
    
Nf=len(AAidx_Dict['C']) #could've just written 15...

pat=re.compile('[\\*_XB]')  ## non-productive CDR3 patterns


C	-8.36918251369427	8.3031934498954	-6.6196694564762	13.8734139891373	8.59531323791867	9.79914818190207	1.26911186662177	-4.63752901352054	-0.983921565990354	4.3634243590938	2.34365256090458	0.343445840449375	1.18290876288827	0.0132558803925318	-0.696621502763574



In [5]:
import pickle
#Save the output of the mapping AAidx_dict in the beginning of DeepCAT.py
with open('AAidx_dict.pkl', 'wb') as f: 
    pickle.dump(AAidx_Dict, f)

In [50]:
np.set_printoptions(precision=3)
AAs=np.array(list('WFGAVILMPYSTNQCKRHDE'))

def OneHotEncoding(Seq):
    Seq_aa = list(Seq) 
    Ns=len(Seq_aa)
    OHE=np.zeros([20,Ns])
    for ii in range(Ns):
        aa=Seq_aa[ii]
        vv=np.where(AAs==aa)
        OHE[vv,ii]=1
    OHE=OHE.astype(np.float32)
    return OHE, vv 
#So in X (row/index) we have the amino acid 
#And in Y (columns) we have the position in the sequence
seq = 'CASSYSTRGGSPLHF'
temp, vv = OneHotEncoding(seq)
print(temp.dtype,"\n", temp.shape,"\n",temp,"\n",type(temp))

float32 
 (20, 15) 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 
 <class 'numpy.nd

In [58]:
print(oneHot_encoding('CASSYSGGSPLHF'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        

In [ ]:
#Nf = number features = 15
Nf=15
def AAindexEncoding(Seq):
    Ns=len(Seq)
    AAE=np.zeros([Ns, Nf])
    for kk in range(Ns):
        ss=Seq[kk]
        #print(ss, kk)
        AAE[kk,]=AAidx_Dict[ss]
    AAE=np.transpose(AAE.astype(np.float32))
    return AAE
seq = 'CASSYSGGSPLHF'
temp = AAindexEncoding(seq)
print(temp,temp.shape)
#print(temp.dtype,"\n", temp.shape,"\n",temp,"\n",type(temp))

In [61]:
llt = torch.tensor([12, 13, 14, 12, 12, 15, 15, 15, 12])

ll = 12
ll1 = 15
x= torch.tensor([1]*len(torch.where(llt==ll)[0])+[0]*len(torch.where(llt==ll1)[0]))
x


tensor([1, 1, 1, 1, 0, 0, 0])

In [71]:
x.size()

torch.Size([7])

In [8]:
def GetFeatureLabels(TumorCDR3s, NonTumorCDR3s):
    nt=len(TumorCDR3s)
    nc=len(NonTumorCDR3s)
    LLt=[len(ss) for ss in TumorCDR3s]
    LLt=np.array(LLt)
    LLc=[len(ss) for ss in NonTumorCDR3s]
    LLc=np.array(LLc)
    NL=range(12,17)
    FeatureDict={}
    LabelDict={}
    for LL in NL:
        vvt=np.where(LLt==LL)[0]
        vvc=np.where(LLc==LL)[0]
        Labels=[1]*len(vvt)+[0]*len(vvc)
        Labels=np.array(Labels)
        Labels=Labels.astype(np.int32)
        data=[]
        for ss in TumorCDR3s[vvt]:
            if len(pat.findall(ss))>0:
                continue
            data.append(AAindexEncoding(ss))
#            data.append(OneHotEncoding(ss))
        for ss in NonTumorCDR3s[vvc]:
            if len(pat.findall(ss))>0:
                continue
            data.append(AAindexEncoding(ss))
#            data.append(OneHotEncoding(ss))
        data=np.array(data)
        features={'x':data,'LL':LL}
        FeatureDict[LL]=features
        LabelDict[LL]=Labels
    return FeatureDict, LabelDict

In [32]:
# get current directory 
path = os.getcwd() 
print("Current Directory", path) 
  
# prints parent directory 
par = os.path.abspath(os.path.join(path, os.pardir))


Current Directory C:\Users\richi\Documents\EPFL\Master\PDM\code\DeepTCR_PyTorch\notebooks


['NormalCDR3.txt',
 'NormalCDR3_test.txt',
 'readme.md',
 'TumorCDR3.txt',
 'TumorCDR3_test.txt']

In [ ]:
import pickle
with open('../AAidx_dict.pkl', 'rb') as f: 
    AAidx_Dict = pickle.load(f) 
def aaindex_encoding(seq):
	"""Encodes the AA indices to a given sequence"""
	n_aa = len(seq)
	temp = np.zeros([n_aa, 15], dtype=np.float32)
	for idx in range(n_aa):
		aa = seq[idx]
		temp[idx] = AAidx_Dict[aa]
	temp = np.transpose(temp)
	aa_encoding = torch.from_numpy(temp)
	return aa_encoding

In [256]:
def read_sequences(filename):
	"""read sequences from a txt, and """
	if '.txt' not in filename:
		print("Non .txt file given, exiting")
		return
		
	data = []
	with open(filename, 'r') as f:
		for line in f.readlines():
			seq = line.strip()
			if not seq.startswith('C') or not seq.endswith('F'):continue
			data.append(seq)
	return np.array(data)


In [260]:
tumor_sequences

array(['CASSGTVGESEQYF', 'CASSWNEGRYGYTF', 'CASTSGTGDEKLFF',
       'CSVAIRNPETQYF', 'CASSRTSGRREQYF', 'CASSLSGGHSNTGELCF',
       'CASTQVDGTDTQYF', 'CASSPQGQGEGGYTF', 'CASSLGTSGRGETQYF',
       'CSATSVSYEQYF', 'CASSQRARNQPQHF', 'CASCSRTGGYTF',
       'CASSRPGQFNEKLFF', 'CASSLPAHYEQYF', 'CAKGGGEGEGILGSPFPLCSDNEQFF',
       'CSASLGLAGYEQFF', 'CASSGDRGTQPQHF', 'CASGLGASYEQYF',
       'CASSFGLAGTSETQYF', 'CASSVAGTANTGELFF', 'CASTSGPEMYQETQYF',
       'CASSETGIGAFF', 'CATSATPDSSPYEQYF', 'CAARRGDTEAFF',
       'CATSHPGTVYYEHYF', 'CASSTRSGMGNEQFF', 'CASSISADTQYF',
       'CASSRTDKEQYF', 'CASSVLRDSYNEQFF', 'CASTLTEVQETQYF',
       'CASSQVTTGQYF', 'CASSSGVRGSSYEQYF', 'CSARSGGPQETQYF',
       'CASSLPDSKLDEQYF', 'CASTTKRDRDNQPQHF', 'CASSQDPVDTQYF',
       'CASSLSAGGTNEQFF', 'CASSPTIGANVLTF', 'CSAYDSYNSPLHF',
       'CSAPGAGGETQYF', 'CASSGGTGAREQDF', 'CASSVVMDNEQFF',
       'CSVADRAFTEAFF', 'CASSAILAGGRSYNEQFF', 'CSASDAGYYEQYF',
       'CASSSFAGPNAGELFF', 'CSAREWGRTDTQYF', 'CASSPHIYGYTF',
       

In [259]:

for filename in os.listdir(par+'/TrainingData/'):
    if '.txt' in filename:
        test = read_sequences(par+'/TrainingData/'+filename)
#DRAFT FOR GENERATE DATA        
tumor_sequences = np.array(test[0:1000])
normal_sequences = np.array(test[1000:5000])
seqlens_tumor = np.array([len(seqs) for seqs in tumor_sequences]) 
seqlens_normal = np.array([len(seqs) for seqs in normal_sequences])
length = 12
mask_tumor = np.where(seqlens_tumor==length)[0]
mask_normal = np.where(seqlens_normal==length)[0]
#Reusing the code from DeepCAT for Labels
labels = torch.tensor(([1]*len(mask_tumor)+[0]*len(mask_normal)), dtype=torch.int64)
len(labels)
data = []
PAT = re.compile('[\\*_XB]')  ## non-productive CDR3 patterns
for seqs in tumor_sequences[mask_tumor]:
    if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
    data.append(aaindex_encoding(seqs))

for seqs in normal_sequences[mask_normal]:
    if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
    data.append(aaindex_encoding(seqs))
data = torch.stack(data)

feature_dict, label_dict = {}, {}
features = {'x':data, 'length':length}
feature_dict[length] = features
label_dict[length] = labels 

In [246]:
def generate_data(tumor_sequences, normal_sequences):
    """For each CDR3 dataset (tumor and normal) sequences, get the feature vectors and labels"""
    
    #Normally, sequences are extracted as lists, but maybe I can modify something in read_sequences to return array instead of list
    if type(tumor_sequences)!=np.ndarray : tumor_sequences = np.array(tumor_sequences)
    if type(tumor_sequences)!=np.ndarray : normal_sequences = np.array(normal_sequences)

    #length of each datapoint (sequence)
    seqlens_tumor = np.array([len(seqs) for seqs in tumor_sequences]) 
    seqlens_normal = np.array([len(seqs) for seqs in normal_sequences])

    feature_dict, label_dict = {}, {}
    #Only keep sequences with length 12 to 16
    for length in range(12,17):
        #Using numpy to create mask for fancy indexing, converting to tensors later
        mask_tumor = np.where(seqlens_tumor==length)[0]
        mask_normal = np.where(seqlens_normal==length)[0]
        #Reusing the code from DeepCAT for Labels
        labels = torch.tensor(([1]*len(mask_tumor)+[0]*len(mask_normal)), dtype=torch.int64)
        data = []

        for seqs in tumor_sequences[mask_tumor]:
            if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
            data.append(aaindex_encoding(seqs))

        for seqs in normal_sequences[mask_normal]:
            if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
            data.append(aaindex_encoding(seqs))
            
        data = torch.stack(data) #Stack a list of tensors into a single tensor
        features = {'x':data, 'length':length}
        feature_dict[length] = features
        label_dict[length] = labels 

    return feature_dict, label_dict

In [247]:
feats, lab = generate_data(tumor_sequences, normal_sequences)

In [252]:
lab[15], feats[15]

(tensor([1, 1, 1,  ..., 0, 0, 0]),
 {'x': tensor([[[-8.3692e+00, -9.7091e-01,  1.1853e+01,  ..., -7.4993e+00,
             4.5303e+00, -1.8596e+01],
           [ 8.3032e+00, -3.2368e-01,  6.8846e+00,  ...,  1.3100e+00,
             5.1265e+00,  9.2236e-01],
           [-6.6197e+00,  1.5721e+01,  2.9673e+00,  ..., -1.2078e+01,
             1.4361e+00, -3.3174e+00],
           ...,
           [ 1.1829e+00,  1.5334e+00,  1.3610e+00,  ...,  7.4468e+00,
             1.9273e-01,  2.9441e-01],
           [ 1.3256e-02,  1.3445e+00,  1.1331e+00,  ...,  3.0783e+00,
            -2.5440e-01, -3.1884e+00],
           [-6.9662e-01,  3.3582e+00,  1.1640e+00,  ...,  2.8487e+00,
            -3.2557e+00, -6.4560e+00]],
  
          [[-8.3692e+00, -9.7091e-01,  1.1853e+01,  ..., -1.7636e+01,
            -1.8596e+01, -1.8596e+01],
           [ 8.3032e+00, -3.2368e-01,  6.8846e+00,  ..., -3.5240e-01,
             9.2236e-01,  9.2236e-01],
           [-6.6197e+00,  1.5721e+01,  2.9673e+00,  ...,  1.1839e+01